# Lab 3: Singing a song

#### Import Data Set

In [423]:
# Import data set

import numpy as np
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")
# find unique values in data set for item
print(xmas["Gift.Item"].unique)
xmas.head()

<bound method Series.unique of 0     partridge
1          dove
2           hen
3          bird
4          ring
5         goose
6          swan
7          maid
8          lady
9          lord
10        piper
11      drummer
Name: Gift.Item, dtype: object>
<bound method Series.unique of 0     partridge
1          dove
2           hen
3          bird
4          ring
5         goose
6          swan
7          maid
8          lady
9          lord
10        piper
11      drummer
Name: Gift.Item, dtype: object>


,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN


#### Function 1: pluralize_gift()

In [434]:
# pluralize_gift()
def pluralize_gift(gift):
    """
  Returns plural of a noun
  
  Parameters
  ----------
  gift: str
    A noun
    
  Return
  ------
  str
    Plural version
  """
  # find plurals of the odd words from unique values
    if isinstance(gift, str):
        if "oo" in gift:
            gift = gift.replace("oo", "ee")
        elif gift[-1] == "y":
          gift = gift.replace("y", "ies")
        else:
            gift = gift + "s"
        return gift

xmas["Plural.Gift.Item"] = xmas["Gift.Item"].apply(pluralize_gift)
xmas.head()

#reference used: https://www.w3schools.com/python/ref_func_isinstance.asp
# reference used: https://www.geeksforgeeks.org/numpy-where-in-python/


,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Day.Number,Plural.Gift.Item
0,1,first,partridge,NaN,NaN,in a pear tree,one,partridges
1,2,second,dove,NaN,turtle,NaN,two,doves
2,3,third,hen,NaN,french,NaN,three,hens
3,4,fourth,bird,NaN,calling,NaN,four,birds
4,5,fifth,ring,NaN,golden,NaN,five,rings


In [435]:
# check function works
pluralize_gift(xmas['Gift.Item'])

#### Function 2: make_phrase()

In [436]:
# mapping numbers to words

number_to_word = {
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
    10: "ten",
    11: "eleven",
    12: "twelve"
}
xmas["Day.Number"] = xmas["Day"].map(number_to_word)


In [502]:
# make_phrase()
import re

def make_phrase(num, num_word, item, verb, adjective, location):
    # Step 1: Replace NAs with blank strings
    verb = pd.Series([verb]).fillna("").iloc[0]
    adjective = pd.Series([adjective]).fillna("").iloc[0]
    location = pd.Series([location]).fillna("").iloc[0]
    # Step 2: If the day number is larger than 1, the gift items need pluralized!
    if num > 1:
        item = pluralize_gift(item)
    # Step 3: Figure out if a gift item starts with a vowel
    vowels = "AEIOUaeiou"
    is_vowel = item[0] in vowels if item else False
    # Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a"
    if num == 1:
        if is_vowel:
            num_word = "an"
        else: num_word = "a"
# Step 5: Put all of the pieces together into one string and return!
    text = " ".join([num_word, adjective, item, verb, location]).strip()
    # get rid of extra spaces
    text = re.sub(r"\s+", " ", text)
    return text

# Reference used: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.index.html

In [503]:
# check to see if function works
print(make_phrase(10, "ten", "lord", "a-leaping", "", ""))

ten lords a-leaping
ten lords a-leaping


In [504]:
# create new column called Full.Phrase and join onto xmas
xmas["Full.Phrase"] = xmas.apply(lambda row: make_phrase(
    row["Day"],
    row["Day.Number"],
    row["Gift.Item"],
    row["Verb"],
    row["Adjective"],
    row["Location"]), axis = 1)
xmas.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Day.Number,Plural.Gift.Item,Full.Phrase
0,1,first,partridge,NaN,NaN,in a pear tree,one,partridges,a partridge in a pear tree
1,2,second,dove,NaN,turtle,NaN,two,doves,two turtle doves
2,3,third,hen,NaN,french,NaN,three,hens,three french hens
3,4,fourth,bird,NaN,calling,NaN,four,birds,four calling birds
4,5,fifth,ring,NaN,golden,NaN,five,rings,five golden rings


#### Function 3: sing_day()

In [513]:
pip install num2words

In [514]:
# function 3: sing_day()
from num2words import num2words

def sing_day(dataset, num, phrase_col):
    num_word = num2words(num, to="ordinal")
    intro = "On the " + num_word + " day of Christmas, my true love gave to me:"
    phrases = dataset.loc[dataset["Day"] <= num, phrase_col].iloc[::-1].tolist()
    phrases = [phrase.strip() for phrase in phrases]
    if len(phrases) > 1:
        song_line = ',\n'.join(phrases[:-1]) + ', and\n' + phrases[-1]
    elif len(phrases) == 1:
        song_line = phrases[0]
    else:
        song_line = ""
    
# strip any extra spaces and get rid of last comma after partridge in a pear tree
    full_song = f'{intro}\n{song_line}.'
    return full_song

In [515]:
#check to see if function works
print(sing_day(xmas, 12, "Full.Phrase"))

On the twelfth day of Christmas, my true love gave to me:
twelve drummers drumming,
eleven pipers piping,
ten lords a-leaping,
nine ladies dancing,
eight maids a-milking,
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.
On the twelfth day of Christmas, my true love gave to me:
twelve drummers drumming,
eleven pipers piping,
ten lords a-leaping,
nine ladies dancing,
eight maids a-milking,
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.


In [516]:
# put the song all together
# print every day
def song_line(dataset, day_col, phrase_col):
    for day in range (1,13):
        print(sing_day(dataset, day, phrase_col))
        # print an empty line inbetween every day
        print("\n")

print(song_line(xmas, "Day", "Full.Phrase"))

On the first day of Christmas, my true love gave to me:
a partridge in a pear tree.


On the second day of Christmas, my true love gave to me:
two turtle doves, and
a partridge in a pear tree.


On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves, and
a partridge in a pear tree.


On the fourth day of Christmas, my true love gave to me:
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.


On the fifth day of Christmas, my true love gave to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.


On the sixth day of Christmas, my true love gave to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.


On the seventh day of Christmas, my true love gave to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, 

In [517]:
# import second dataset
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
# mapping numbers to words

number_to_word = {
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
    10: "ten",
    11: "eleven",
    12: "twelve"
}
xmas2["Day.Number"] = xmas2["Day"].map(number_to_word)
# create new column called Full.Phrase and join onto xmas
xmas2["Full.Phrase"] = xmas2.apply(lambda row: make_phrase(
    row["Day"],
    row["Day.Number"],
    row["Gift.Item"],
    row["Verb"],
    row["Adjective"],
    row["Location"]), axis = 1)

In [518]:
print(song_line(xmas2, "Day", "Full.Phrase"))

On the first day of Christmas, my true love gave to me:
an email from Cal Poly.


On the second day of Christmas, my true love gave to me:
two meal points, and
an email from Cal Poly.


On the third day of Christmas, my true love gave to me:
three lost pens,
two meal points, and
an email from Cal Poly.


On the fourth day of Christmas, my true love gave to me:
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.


On the fifth day of Christmas, my true love gave to me:
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.


On the sixth day of Christmas, my true love gave to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.


On the seventh day of Christmas, my true love gave to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.

### Appendix and References

https://www.w3schools.com/python/ref_func_isinstance.asp

https://www.geeksforgeeks.org/numpy-where-in-python/ 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.index.html

Generative A.I. Statement: Chat-GPT was used to suggest changes in code to debug errors. An example of errors include, incorrect syntax, incorrect usage of parameters(if, else format) for functions, making suggestions to fix parameters to resolve the error in code, and also understanding how to use pd.Series(), isinstance, np.where(). Chat-GPT was only used to resolve errors in already hand written code. 